# Colosseum ticket availability webscraper

## Written by Nicholas Fasano
## Created: 04/09/2023
### Description: Repeatedly query Colosseum website for ticket availability. This script is optimized for tracking ticket availability over the course of the month since they are released. 
### Website: https://ecm.coopculture.it

In [ ]:
# Load in python packages
import numpy as np
import pandas as pd

# for time and date manipulations
import datetime
import time

# load in Selenium functionalities
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# import sending email functionality
import smtplib, ssl

# path to chrome webdriver - will need to install to local machine based on you version of Chrome
# See: https://chromedriver.chromium.org/downloads
PATH_CHROME = "C:\Program Files (x86)\ManuallyInstalled\chromedriver_v111.exe"

# User defined functions used in webscraping https://ecm.coopculture.it

In [ ]:
def table_indices(date):
    """
    Input: date: string in 'mm/dd/yyyy' format
    Output: row (week number) and col (day of week) that date belongs to
    Assumes that the calendar column format is 'S M T W Th F S'
    Function adapted from: 'https://stackoverflow.com/questions/3806473/week-number-of-the-month#:~:text=The%20answer%20you%20are%20looking,and%20%25%20is%20the%20positive%20remainder.&text=As%20dm%20and%20dw%20are,)%257)%2F7%20%2B%201%20'
    """
    
    # convert input date from string format to datetime object
    date = datetime.datetime.strptime(date, '%m/%d/%Y')
    day_of_week = date.weekday()
    

    # Compute day number (column)
    # adjust output from .weekday() which outputs Monday = 0, Tuesday = 1, ... Sunday = 6
    # so that Sunday = 1, Monday = 2, ... Saturday = 7
    if(day_of_week == 6):
        # first day of the month is a Sunday -- set to 1
        col = 1
    else:
        # day is Monday - Saturday, increment .weekday() output by 2
        col = day_of_week + 2
    
    # compute week number (row)
    
    first_day = date.replace(day = 1).weekday()
    
    # adjust output from .weekday() which outputs Monday = 0, Tuesday = 1, ... Sunday = 6
    # so that Sunday = 0, Monday = 1, ... Saturday = 6
    if(first_day == 6):
        # first day of the month is a Sunday -- set to 0
        first_day = 0
    else:
        # first day is Monday - Saturday, increment .weekday() output by 1
        first_day = first_day + 1
    day_of_month = date.day + first_day
    
    # add 2 to account for month label and day of week label in the html table
    # the +2 is specific to the colosseum website calendar and may vary for other types of calendars
    row = int(np.ceil(day_of_month/7.0)) + 2 

    return row, col 

def calendar_day_color(day_color):
    # input: day_color is the text of the html element that is associated with the color coding of the calendar day
    
    if(day_color == 'background: rgb(206, 234, 208);'):
        return 'green' # tickets available
    elif(day_color == 'background: rgb(255, 224, 223);'):
        return 'red'   # tickets soldout
    elif(day_color == 'background: rgb(238, 238, 238);'):
        return 'grey'  # date unavailable (grey day -- either day has passed or tickets have not been released yet)
    else:
        return 'white' # tickets not being sold online for this day (white)

# List urls to different types of tickets

In [ ]:
# choose which ticket types you are interested in tracking based on the url to that ticket availability calendar

# Full experience ticket (underground + arena)
url1 = 'https://ecm.coopculture.it/index.php?option=com_snapp&view=event&id=D7E12B2E-46C4-074B-5FC5-016ED579426D&catalogid=DDDA3AB3-47BC-0A49-7752-0174490F632A&lang=en'

# Full experience ticket (arena only)
url2 = 'https://ecm.coopculture.it/index.php?option=com_snapp&view=event&id=3C38AB77-8D5A-5394-05B2-0172EB8E7D46&catalogid=F3CB77BD-6A43-108B-6723-0174490EB610&lang=en'

# Full eperience with english didactic tour
url3 = 'https://ecm.coopculture.it/index.php?option=com_snapp&view=event&id=490E25D6-2465-ED3A-6A13-016ED583FB68&catalogid=238D971A-D296-07C6-7A82-0174490F9C7B&lang=en'

# Regular entrance 
url4 ='https://ecm.coopculture.it/index.php?option=com_snapp&view=event&id=3793660E-5E3F-9172-2F89-016CB3FAD609&catalogid=B79E95CA-090E-FDA8-2364-017448FF0FA0&lang=en'

# Regular entrance plus english tour
url5 ='https://ecm.coopculture.it/index.php?option=com_snapp&view=event&id=96DBBFE1-BC45-FDCB-5E1B-016CF138154A&catalogid=7049E852-9020-9834-008D-017FD631028C&lang=en'

# Colosseum by night (english tour)
url6  = 'https://ecm.coopculture.it/index.php?option=com_snapp&view=event&id=2E4CF2E0-F5B7-7757-4241-0173584374D6&catalogid=06B97E06-CA46-981E-360A-017A9FE4C3F6&lang=en'

# Regular entrance for groups and schools
url7 ='https://ecm.coopculture.it/index.php?option=com_snapp&view=event&id=1C12B646-0675-43E4-6480-016CDCFD059A&catalogid=343A9780-C74C-B357-316E-01750CD2B9BA&lang=en'


# choose urls to track
urls = [url1,url2,url3,url4,url5,url6]

# name_urls == unique string identifier for tracked ticket types -- saved in dataframe index for filtering purposes
name_urls = ['FEUA','FEA','FEUAG','RE','REG','NIGHT']

# Set control parameters for monitoring ticket availability

In [ ]:

# choose date to search for tickets
start_date = '04/16/2023'
end_date = '06/15/2023'

# choose ticket types to track based on urls from above
urls = [url1,url2,url3]

# name_urls == unique string identifier for tracked ticket types -- saved in dataframe index for filtering purposes
name_urls = ['FEUA','FEA','FEUAG']

# choose time window that you want to track tickets (military time used throughout)
# enter your local time (NOT ROME time [CET])
start_time = -1. # start_time < 0 will allow tracking immediately
end_time   = 18.5 #  end_time > 23 will track through midnight
time_between_calls = .2 # in seconds

# set the number of iterations before saving dataframe to pickle file
iter_between_save  = 10 

# pickle file saved as: current_date + "_all_days_" + file_extension + ".pkl"
file_extension = 'v3'

# Initialize pandas dataframe for recording webscraped data

In [ ]:
# create ticket time list: ticket times are in intervals of 5 minutes from 9am-10pm (Rome time [CET])
time_list = [str(i*datetime.timedelta(minutes=5)) for i in range(9*60//5,23*60//5)]
time_list = ['0' + time[:-3] if ':' in time[0:2] else time[:-3] for time in time_list ]
time_list.insert(0,'Color') # track color of calendar day

start_date = datetime.datetime.strptime(start_date,'%m/%d/%Y')   # start date
end_date = datetime.datetime.strptime(end_date,'%m/%d/%Y')   # end date
date_list = list(pd.date_range(start_date,end_date-datetime.timedelta(days=1),freq='d').strftime('%m/%d/%Y'))

indices = [date_list, time_list]
columns = pd.MultiIndex.from_product(indices, names=["day", "entryTime"])

data = {columns[j]:pd.arrays.SparseArray(np.nan,fill_value=np.nan) for j in range(len(columns))}
df = pd.DataFrame(data = data,columns=columns,index=[datetime.datetime.now().strftime('%m/%d/%Y_%H:%M:%S')])

# Begin primary loop to webscrape ticket availability across all days where in range start_date through end_date

In [ ]:
# hold off on making webpage calls until start_time -- tickets go live at 3am
current_time = float(datetime.datetime.now().strftime("%H"))
while(current_time < start_time or current_time > end_time):
    time.sleep(120)
    current_time = float(datetime.datetime.now().strftime("%H"))
    
    
# open webpage and format screensize
driver = webdriver.Chrome(PATH_CHROME)
driver.get(url1)
driver.maximize_window()
driver.execute_script("window.scrollTo(0, 500)")
time.sleep(5)

# first loop over certain timespan
current_time = float(datetime.datetime.now().strftime("%H"))
save_counter = 0 # iteration tracker to save dataframe every ten calls

while(current_time > start_time or current_time < end_time):
    # second loop over number of ticket types you want to track
    for jurl in range(len(urls)):
        driver.get(urls[jurl])
        driver.execute_script("window.scrollTo(0, 500)")
        time.sleep(3)
        
        ticket_time = []
        number_avail = []      
        search_time = datetime.datetime.now().strftime("%m/%d/%Y_%H:%M:%S") + '_' + name_urls[jurl] + '_'
        
        # third loop over months
        for jmonth in range(2):

            # find color of all days in current month                                                            
            elements = driver.find_elements(By.XPATH,"//td[@class = 'calendar-day dayactive text-center']") + driver.find_elements(By.XPATH,"//td[@class = 'calendar-day  text-center']")

            # final loop over all days in that month
            for el in elements:

                # assign color to all dates, if color green (get availability too)
                date_text = datetime.datetime.strptime(el.find_element(By.TAG_NAME,"div").get_attribute('data-date'), "%d/%m/%Y").strftime("%m/%d/%Y")
                day_color = calendar_day_color(el.get_attribute('style')) # color
                if(date_text in date_list):
                    if(day_color == 'green'):
                        # generate days link
                        row, col = table_indices(date_text)
                        search_date_str = "//*[@id='calendar']/table/tbody/tr[" + str(row) + "]/td[" + str(col) +"]"

                        # click desired day
                        element_click_date = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, search_date_str + "/div")))
                        element_click_date.click()
                        time.sleep(3)

                        # check if 'ATTENTION:\n No result is available day' message is presented
                        element_no_avail = driver.find_elements(By.XPATH,'//*[@id="msg"]')

                        if(len(element_no_avail) < 1):
                            # get ticket time and availability
                            el_ticket_time = driver.find_elements(By.XPATH,"//div[@class='col-md-4 col-sm-4 col-xs-4']")
                            el_number_avail = driver.find_elements(By.XPATH,"//div[@class='col-md-8 col-sm-8 col-xs-8 nopadding']")

                            ticket_time = ticket_time + [(date_text, jtime.text) for jtime in el_ticket_time]

                            number_avail_temp = [jnum_avail.text for jnum_avail in el_number_avail]
                            number_avail = number_avail + [int(text[text.find('(')+1:text.find(')')]) for text in number_avail_temp]

                            # check if there are more than 10 available ticket times, if yes then iterate through all times
                            num_pages = len(driver.find_elements(By.XPATH,"//a[@class = 'page-link']"))
                            for k in range(num_pages):
                                driver.find_element(By.XPATH,"//a[@class = 'page-link next']").click()
                                time.sleep(4)
                                
                                # get ticket time and availability
                                el_ticket_time = driver.find_elements(By.XPATH,"//div[@class='col-md-4 col-sm-4 col-xs-4']")
                                el_number_avail = driver.find_elements(By.XPATH,"//div[@class='col-md-8 col-sm-8 col-xs-8 nopadding']")

                                ticket_time = ticket_time + [(date_text, jtime.text) for jtime in el_ticket_time]

                                number_avail_temp = [jnum_avail.text for jnum_avail in el_number_avail]
                                number_avail = number_avail + [int(text[text.find('(')+1:text.find(')')]) for text in number_avail_temp]

                            ticket_time.append((date_text,'Color'))
                            number_avail.append(day_color)
                            
                        else: # tickets sold out
                            ticket_time = ticket_time + [(date_text,'Color')]
                            number_avail = number_avail + [day_color]

                    else: # no availability
                        ticket_time = ticket_time + [(date_text,'Color')]
                        number_avail = number_avail + [day_color]

            # advance the month
            if(jmonth < 1):
                advance_month = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//*[@id='calendar']/table/tbody/tr[1]/td[3]/div")))
                advance_month.click(); 
                time.sleep(4)

        df.loc[search_time] = {ticket_time[j]:number_avail[j] for j in range(len(ticket_time))}   

    if(save_counter%iter_between_save == 0):
        current_date = datetime.datetime.now().strftime("%m_%d_%Y")
        df.to_pickle(current_date + "_all_days_" + file_extension + ".pkl")  
    save_counter = save_counter + 1
    time.sleep(time_between_calls)
    
    current_time = float(datetime.datetime.now().strftime("%H"))

# Finally save dataframe and then close driver
current_date = datetime.datetime.now().strftime("%m_%d_%Y")
df.to_pickle(current_date + "_all_days_" + file_extension + ".pkl")  
driver.close()